In [2]:
import pandas as pd   
import numpy as np

from bs4 import BeautifulSoup 
import re
from nltk.corpus import stopwords # Import the stop word list
from nltk.stem.porter import PorterStemmer

def clean_review(raw_review):
	example1 = BeautifulSoup(raw_review, 'html.parser').get_text()
	# print type(example1)
	# Use regular expressions to do a find-and-replace
	letters_only = re.sub("[^a-zA-Z]",           # The pattern to search for
	                      " ",                   # The pattern to replace it with
	                      example1 )  # The text to search
	# print letters_only
	lower_case = letters_only.lower()        # Convert to lower case
	words = lower_case.split()               # Split into words
	# print words
	# print stopwords.words("english")
	words = [w for w in words if not w in stopwords.words("english")]
	# print words
	from nltk.stem.porter import PorterStemmer
	porter_stemmer = PorterStemmer()
	stemmed_words = []
	for i in range(len(words)):
		stemmed_words.append(porter_stemmer.stem(words[i]))
	return( " ".join( stemmed_words ))


train = pd.read_csv("data\IMDB\labeledTrainData.tsv", header=0, delimiter="\t", quoting=3)
print(train.shape)
# print train.columns.values
num_reviews = train["review"].size
clean_reviews = []
for i in range(10):
	if( (i+1)%1000 == 0 ):
		print("Review %d of %d\n" % ( i+1, num_reviews )   )                                                       
	clean_reviews.append(clean_review(train["review"][i]))
%store clean_reviews

(25000, 3)


Stored 'clean_reviews' (list)


In [5]:
print(len(clean_reviews))


for i in range(len(clean_reviews)):
    clean_reviews[i] = clean_reviews[i].split(' ')


print(clean_reviews[1])

10
['classic', 'war', 'world', 'timothi', 'hine', 'entertain', 'film', 'obvious', 'goe', 'great', 'effort', 'length', 'faith', 'recreat', 'h', 'g', 'well', 'classic', 'book', 'mr', 'hine', 'succe', 'watch', 'film', 'appreci', 'fact', 'standard', 'predict', 'hollywood', 'fare', 'come', 'everi', 'year', 'e', 'g', 'spielberg', 'version', 'tom', 'cruis', 'slightest', 'resembl', 'book', 'obvious', 'everyon', 'look', 'differ', 'thing', 'movi', 'envis', 'amateur', 'critic', 'look', 'critic', 'everyth', 'other', 'rate', 'movi', 'import', 'base', 'like', 'entertain', 'peopl', 'never', 'agre', 'critic', 'enjoy', 'effort', 'mr', 'hine', 'put', 'faith', 'h', 'g', 'well', 'classic', 'novel', 'found', 'entertain', 'made', 'easi', 'overlook', 'critic', 'perceiv', 'shortcom']


In [33]:
clean_test_reviews = [] 


for i in range(10,15):
    
    clean_test_reviews.append( clean_review( train["review"][i] ))
    
test_op = train["sentiment"][10:15]
print(test_op)

10    0
11    1
12    1
13    0
14    0
Name: sentiment, dtype: int64


In [14]:
from lib.tensorflowglove import tf_glove
glovemodel = tf_glove.GloVeModel(embedding_size=300, context_size=10)
glovemodel.fit_to_corpus(clean_reviews)

In [15]:
glovemodel.train(num_epochs=10)

In [ ]:
print(type(test_op))
print(clean_test_reviews[5])

<class 'pandas.core.series.Series'>


IndexError: list index out of range

In [16]:
import matplotlib
%matplotlib inline
glovemodel.generate_tsne()

In [17]:
glovemodel.embeddings

array([[-0.55191505, -0.58698887, -0.2480624 , ..., -0.532812  ,
        -0.47619271, -0.3020153 ],
       [-1.05818248,  0.29258564, -1.53566766, ...,  0.91010624,
        -0.06010503, -0.03974052],
       [-0.07453331,  0.55578732, -0.39850479, ...,  0.16432957,
         0.258349  , -0.3587175 ],
       ..., 
       [ 0.06615934,  0.74716002,  0.2909618 , ...,  1.30511546,
        -1.29501677,  0.0318346 ],
       [-0.14203125,  0.38812864,  0.34969291, ...,  1.00333774,
        -0.32389164, -0.65898448],
       [ 0.45654786, -0.40532488, -0.06118235, ...,  0.92906642,
        -1.06934106, -1.55059385]], dtype=float32)

In [22]:
glovemodel.embeddings[glovemodel.id_for_word('k')]

array([-0.03222942, -0.36123627, -0.08965302, -0.87305146,  0.10418586,
       -0.82008815, -0.26724911,  1.11199832, -0.82340115, -0.44559732,
       -0.30026934,  1.0071888 , -0.13929728, -0.62022358, -0.85671866,
        0.78747004, -0.90620524,  0.7232579 , -0.13922298,  1.06302559,
       -0.40877268,  1.02106941, -1.13613915,  0.50750464, -0.81685674,
        0.03576756, -0.19330762, -0.2572692 ,  0.02214804,  0.06502384,
       -0.70460033,  0.52478373,  0.76222306,  0.31561702,  1.06144416,
        0.78358477,  0.20295665,  1.22286308,  0.75359458, -1.78022742,
       -0.55827641, -0.22303838, -0.15053667, -1.64959645,  0.81298316,
        0.4726342 , -0.06501664, -0.35705787,  0.66635913, -0.15118098,
       -1.36974335, -0.3380996 ,  1.25974858,  0.45577219,  1.04450655,
       -0.25281462,  1.54658175, -0.00569487,  0.40391719,  0.82098067,
       -0.73775345, -0.66108769, -0.67710793, -1.08653831,  1.14112234,
       -0.94671381, -0.71269953, -0.8652851 , -0.42442209,  1.14

In [19]:
print(glovemodel.max_vocab_size)

100000


In [20]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM

In [51]:
max_features = 20000
maxlen = 150  # cut texts after this number of words (among top max_features most common words)
batch_size = 32

In [52]:
print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(clean_reviews, maxlen=maxlen)
x_test = sequence.pad_sequences(clean_test_reviews, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Pad sequences (samples x time)


ValueError: invalid literal for int() with base 10: 'call'

In [29]:
model = Sequential()
model.add(Embedding(glovemodel.max_vocab_size, 300))
model.add(LSTM(300, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

# try using different optimizers and different optimizer configs

In [30]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [53]:
print('Train...')
model.fit(x_train, np.array(train["sentiment"][0:10]),
          batch_size=batch_size,
          epochs=15,
          validation_data=(x_test,np.array(test_op)))

Train...


NameError: name 'x_train' is not defined

In [ ]:
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

In [50]:
print(np.array(train["sentiment"][0:10])[0])
print((clean_reviews[1]))

1
['classic', 'war', 'world', 'timothi', 'hine', 'entertain', 'film', 'obvious', 'goe', 'great', 'effort', 'length', 'faith', 'recreat', 'h', 'g', 'well', 'classic', 'book', 'mr', 'hine', 'succe', 'watch', 'film', 'appreci', 'fact', 'standard', 'predict', 'hollywood', 'fare', 'come', 'everi', 'year', 'e', 'g', 'spielberg', 'version', 'tom', 'cruis', 'slightest', 'resembl', 'book', 'obvious', 'everyon', 'look', 'differ', 'thing', 'movi', 'envis', 'amateur', 'critic', 'look', 'critic', 'everyth', 'other', 'rate', 'movi', 'import', 'base', 'like', 'entertain', 'peopl', 'never', 'agre', 'critic', 'enjoy', 'effort', 'mr', 'hine', 'put', 'faith', 'h', 'g', 'well', 'classic', 'novel', 'found', 'entertain', 'made', 'easi', 'overlook', 'critic', 'perceiv', 'shortcom']
